In [ ]:
import os
import sys
import json
from collections import Counter
from scipy.integrate import solve_ivp
from scipy.stats import ttest_rel
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from math import pi
import pickle
import matplotlib.pyplot as plt
from matplotlib import rcParams
from packages import data_container
from packages.data_container import Data
from packages.helper import play_trajs, rotate, sp2a, v2sp, dist, psi, beta, d_theta, d_psi, sp2v, dist, \
    traj_speed, min_sep
from packages.ode_simulator import ODESimulator
from packages.models import cohen_avoid_heading
from sklearn.metrics import mean_squared_error
from fitting import Bai_movObst1, Cohen_movObst1, Cohen_movObst2
from parameters import approaches, avoidances

# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

data = None
def load_data(filename):
    global data
    filename = filename + '.pickle'
    file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', filename))
    with open(file, 'rb') as f:
        data = pickle.load(f)

# file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Cohen_movObst1_data_30Hz.pickle'))
# with open(file, 'rb') as f:
#     data = pickle.load(f)


In [ ]:
'''Simulation with var0'''
%matplotlib qt
Hz = 100
xg0, yg0, xo0, yo0, x0, y0 = 0, 10, 5, 5, 0, 0
vxo0, vyo0, vx0, vy0 = -1, 0, 0, 0.9
s0, phi0 = v2sp([vx0, vy0])
a0 = dphi0 = ds0 = 0
w = 0.1
# xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, a, phi, s, dphi, ds = var0
var0 = [xg0, yg0, xo0, yo0, vxo0, vyo0, x0, y0, vx0, vy0, a0, phi0, s0, dphi0, ds0, w]
fajen_approach['ps'] = 1
fajen_approach2['ps'] = 1
cohen_avoid2['ps'] = 1
models = [fajen_approach2, cohen_avoid3]
dw = [0.1] * 1 * Hz + [0] * 9 * Hz
args = {'w_goal': 0.1, 'w_obst': 0.1, 'dw': 0}
sim = ODESimulator(Hz=Hz, models=models, args=args, ref=[0, 1])
sim.simulate(var0, total_time=10)
sim.play()

In [ ]:
'''Simulate one trial approach'''
############
i_trial = 0
approach = approaches['Bai_movObst1b']['fajen_approach']['differential_evolution'][0]
avoid = None
############
%matplotlib qt
models = [approach]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='stimuli_onset', t_end='stimuli_out', ps='trial')
print(sim.test('p_dist'))
sim.play(obst=False)

### Cohen_movObst1

In [ ]:
'''Simulate one trial avoidance'''
load_data('Cohen_movObst1_data')
############
i = 265
approach = approaches['Fajen_steer1a']['fajen_approach2']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid4']['differential_evolution']['-1obst_onset']
# avoid = avoidances['Cohen_movObst1']['cohen_avoid_heading']['differential_evolution'][-1]
model_name = 'Avoidance model 1'
############
%matplotlib qt
models = [approach, avoid]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i], t_start='obst_onset', t_end='obst_out', ps='trial')
dist_error = sim.test('p_dist')
heading_error = sim.test('phi_RMSE') * 180 / np.pi
speed_error = sim.test('s_RMSE')
t_length = (sim.data.info['obst_out'][i] - sim.data.info['obst_onset'][i]) / sim.data.Hz
title =  model_name + \
        ', subj ' + str(sim.data.info['subj_id'][i]) + \
        ', trial ' + str(sim.data.info['trial_id'][i]) + \
        ', obst path angle: ' + str(sim.data.info['obst_angle'][i]) + '°' + \
        ', obst speed: ' + str(sim.data.info['obst_speed'][i]) + ' m/s' + \
        ',\n distance error: ' + str(round(dist_error, 2)) + ' m' + \
        ', heading RMSE: ' + str(round(heading_error, 2)) + '°' + \
        ', speed RMSE: ' + str(round(speed_error, 2)) + ' m/s'
print(f'Simulation time {t_length} s, distance error {dist_error} m')
sim.play(linestyles=['--', '-', '-', '-'], plot=True, fontsize=13, xrange=[-3, 3], yrange=[-1, 9],
         title=title, fig_size=[10, 3.5], save=False)

# plt.gcf().get_axes()[0].legend(loc='lower right')

In [ ]:
'''Simuate all trials model 1'''
load_data('Cohen_movObst1_data')
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid']['differential_evolution']['-1obst_onset']

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Cohen_movObst1(subjects)
sim1.reset()
sim1.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim1.simulate_all(trials1, t_start='obst_onset', t_end='obst_out', ps='trial')
errors1 = sim1.test('p_dist', all_errors=True)
print(f"distance error is {np.mean(errors1)} sd is {np.std(errors1)}")
print(f"order accuracy is {sim1.test('order_accuracy')}")
print(f"heading RMSE is {sim1.test('phi_RMSE')}")
print(f"speed RMSE is {sim1.test('s_RMSE')}")
errors1 = sim1.test('p_dist', all_errors=True)
# Compute BIC
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x**2 for x in errors1]
# n = len(mses)
# n is the n_subj * n_conditions
n_angle = len([x for x in set(data.info['obst_angle']) if x and abs(x) != 0])
n_speed = len([x for x in set(data.info['obst_speed']) if x and abs(x) != 0])
n = len(set(data.info['subj_id'])) * n_angle * n_speed
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim1.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Simuate all trials model 2'''
load_data('Cohen_movObst1_data')
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach2']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst1']['cohen_avoid4']['differential_evolution']['-1obst_onset']

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim2, trials2 = Cohen_movObst1(subjects)
sim2.reset()
sim2.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim2.simulate_all(trials2, t_start='obst_onset', t_end='obst_out', ps='trial')
errors2 = sim2.test('p_dist', all_errors=True)
print(f"distance error is {np.mean(errors2)} sd is {np.std(errors2)}")
print(f"order accuracy is {sim2.test('order_accuracy')}")
print(f"heading RMSE is {sim2.test('phi_RMSE')}")
print(f"speed RMSE is {sim2.test('s_RMSE')}")
errors2 = sim2.test('p_dist', all_errors=True)
# Compute BIC
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x**2 for x in errors2]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim2.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Compare two simulations'''
%matplotlib qt
# plt.plot(errors1 - errors2)
for i in range(len(errors1)):
    if errors1[i] - errors2[i] > 0.1 and 0.1 < errors2[i] < 0.15:
        print(sim1.i_trials[i])

In [ ]:
'''Cross validation error'''
load_data('Cohen_movObst1_data')
def cross_validate(folder):
    path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Results', folder))
    filenames = [os.path.join(path, name) for name in os.listdir(path) if name[-3:] == 'txt']
    errors = {}
    for filename in filenames:
        with open(filename, 'rb') as f:
            best = 0
            e_min = float('inf')
            if filename[-6] == '_' or filename[-6] == '-':
                subj_id = int(filename[-5])
            elif filename[-6] == 'l':
                subj_id = -1
            else:
                subj_id = int(filename[-6:-4])
            for i, line in enumerate(f):
                if i > 10:
                    try:
                        err = str(line).split("\\t")[2][:10]
                        if err[0] == '0':
                            err = float(err)
                            if err < e_min:
                                e_min = err
                                best = str(line).replace("\\", "")
                    except:
                        pass
        try:
            if 'acceleration' in path:
                models = json.loads(''.join(best.split('t')[1] + best.split('t')[2]).replace("\'", "\"")) # If contains acceleration model
                models[0]['name'] = "acceleration_approach" # If contains acceleration model
            else:
                models = json.loads(best.split('t')[1].replace("\'", "\"")) # If not contains acceleration model
            subjects = [int(subj_id)]
            sim, trials = Cohen_movObst1(subjects)
            sim.reset()
            sim.models = models
            sim.simulate_all(trials, t_start='obst_onset', t_end='obst_out', ps='trial')
            errors[subj_id] = sim.test('p_dist', all_errors=True)
            print(f'{subj_id} finished')
            print(f'filename {filename}')
            print(f'models {sim.models}')
            print(f'error {np.mean(errors[subj_id])}')
        except Exception as e:
            print(e)
    return errors

    
errors = cross_validate('Cohen_movObst1_all_all_models_obst_onset')
# Compute BIC
for key, val in errors.items():
    print(key, np.mean(val))
'''BIC = n*ln(MSE) + k*ln(n))'''
mses = [x for a in errors.values() for x in a]
mses = [x**2 for x in mses]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Output errors for analysis in R'''
with open('Cohen_movObst1_CV_errors_obst_onset.csv', 'a') as f:
    f.write('model,subject,error\n')
for i in ['', '2', '3', '4']:
    model = 'cohen_avoid' + i
    errors = cross_validate('Cohen_movObst1_cross_validation_cohen_avoid' + i)
    with open('Cohen_movObst1_CV_errors_obst_onset.csv', 'a') as f:
        for subj, error in errors.items():
            for err in error:
                log = [model, subj, err]
                log = [str(x) for x in log]
                f.write(','.join(log) + '\n')
            

In [ ]:
'''Minimum Passing Distance'''
%matplotlib qt
load_data('Cohen_movObst1_data')
sim = sim1
fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.5), constrained_layout=True)
ax.set_title('Avoidance Model 1')
ax.set_ylabel('SMPD (m)')
ax.set_xlabel('normalized time (%)')
ax.set_ylim((-3, 3))
last_smpd_front = []
last_smpd_behind = []
for j, i in enumerate(sim.i_trials):
    t0 = data.info['obst_onset'][i]
    t1 = data.info['obst_out'][i]
    p0 = sim.p_pred[j]
    p1 = data.info['p_obst'][i][t0:t1]
    v0 = sim.v_pred[j]
    v1 = data.info['v_obst'][i][t0:t1]
    t = np.linspace(0, 100, len(p0))
    smpd = []
    for _p0, _p1, _v0, _v1 in zip(p0, p1, v0, v1):        
        smpd.append(min_sep(_p0, _p1, _v0, _v1)[0])
    ax.plot(t, smpd, 'k', linewidth=0.1, alpha=0.5)
    if smpd[-1] > 0:
        last_smpd_front.append(smpd[-1])
    else:
        last_smpd_behind.append(smpd[-1])
print(np.mean(last_smpd_front), np.std(last_smpd_front), np.mean(last_smpd_behind), np.std(last_smpd_behind))

### Cohen_movObst2

In [ ]:
'''Simulate one trial avoidance'''
load_data('Cohen_movObst2_data')
############
i = 40
approach = approaches['Fajen_steer1a']['fajen_approach2']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst2']['cohen_avoid4']['differential_evolution']['-1obst_onset']
# avoid = avoidances['Cohen_movObst2']['cohen_avoid_heading']['differential_evolution'][-1]
model_name = 'Avoidance model 4'
############
%matplotlib qt
models = [approach, avoid]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i], t_start='obst_onset', t_end='obst_out', ps='trial')
dist_error = sim.test('p_dist')
heading_error = sim.test('phi_RMSE') * 180 / np.pi
speed_error = sim.test('s_RMSE')
t_length = (sim.data.info['obst_out'][i] - sim.data.info['obst_onset'][i]) / sim.data.Hz
title = model_name + \
        ', subj ' + str(sim.data.info['subj_id'][i]) + \
        ', trial ' + str(sim.data.info['trial_id'][i]) + \
        ', obst path angle: ' + str(sim.data.info['obst_angle'][i]) + '°' + \
        ', obst init dist: ' + str(sim.data.info['obst_dist'][i]) + 'm' + \
        ',\n distance error: ' + str(round(dist_error, 2)) + ' m' + \
        ', heading RMSE: ' + str(round(heading_error, 2)) + '°' + \
        ', speed RMSE: ' + str(round(speed_error, 2)) + ' m/s'
print(f'Simulation time {t_length} s, distance error {dist_error} m')
sim.play(linestyles=['--', '-', '-', '-'], plot=True, fontsize=13, xrange=[-3, 3], yrange=[-1, 9],
        title=title, fig_size=[10, 3.5], save=False)


In [ ]:
'''Simuate all trials model 1'''
load_data('Cohen_movObst2_data')
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst2']['cohen_avoid2']['differential_evolution']['-1obst_onset']

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Cohen_movObst2(subjects)
sim1.reset()
sim1.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim1.simulate_all(trials1, t_start='obst_onset', t_end='obst_out', ps='trial')
errors1 = sim1.test('p_dist', all_errors=True)
print(f"distance error is {np.mean(errors1)} sd is {np.std(errors1)}")
print(f"order accuracy is {sim1.test('order_accuracy')}")
print(f"heading RMSE is {sim1.test('phi_RMSE')}")
print(f"speed RMSE is {sim1.test('s_RMSE')}")
errors1 = sim1.test('p_dist', all_errors=True)
# Compute BIC
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x**2 for x in errors1]
# n = len(mses)
# n is the n_subj * n_conditions
n_angle = len([x for x in set(data.info['obst_angle']) if x])
n_dist = len([x for x in set(data.info['obst_dist']) if x])
n = len(set(data.info['subj_id'])) * n_angle * n_dist
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim1.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Simuate all trials model 2'''
load_data('Cohen_movObst2_data')
############
# subject = 1
approach = approaches['Fajen_steer1a']['fajen_approach2']['differential_evolution'][-1]
avoid = avoidances['Cohen_movObst2']['cohen_avoid4']['differential_evolution']['-1obst_onset']

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim2, trials2 = Cohen_movObst2(subjects)
sim2.reset()
sim2.models = models

# for i in range(len(data.trajs)):
#     if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
#         continue
#     trials.append(i)
sim2.simulate_all(trials2, t_start='obst_onset', t_end='obst_out', ps='trial')
errors2 = sim2.test('p_dist', all_errors=True)
print(f"distance error is {np.mean(errors2)} sd is {np.std(errors2)}")
print(f"order accuracy is {sim2.test('order_accuracy')}")
print(f"heading RMSE is {sim2.test('phi_RMSE')}")
print(f"speed RMSE is {sim2.test('s_RMSE')}")
errors2 = sim2.test('p_dist', all_errors=True)
# Compute BIC
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
mses = [x**2 for x in errors2]
# n = len(mses)
# n is the n_subj * n_conditions
n_angle = len([x for x in set(data.info['obst_angle']) if x])
n_dist = len([x for x in set(data.info['obst_dist']) if x])
n = len(set(data.info['subj_id'])) * n_angle * n_dist
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim2.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Compare two simulations'''
%matplotlib qt
# plt.plot(errors1 - errors2)
for i in range(len(errors1)):
    if 0 < errors1[i] - errors2[i] > 0.05 and 0.15< errors1[i] < 0.23:
        print(sim1.i_trials[i])

In [ ]:
load_data('Cohen_movObst2_data')
[x for x in set(data.info['obst_dist']) if x]

In [ ]:
'''Cross validation error'''
load_data('Cohen_movObst2_data')
def cross_validate(folder):
    path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Results', folder))
    filenames = [os.path.join(path, name) for name in os.listdir(path) if name[-3:] == 'txt']
    errors = {}
    for filename in filenames:
        with open(filename, 'rb') as f:
            best = 0
            e_min = float('inf')
            if filename[-6] == '_' or filename[-6] == '-':
                subj_id = int(filename[-5])
            elif filename[-6] == 'l':
                subj_id = -1
            else:
                subj_id = int(filename[-6:-4])
            for i, line in enumerate(f):
                if i > 10:
                    try:
                        err = str(line).split("\\t")[2][:10]
                        if err[0] == '0':
                            err = float(err)
                            if err < e_min:
                                e_min = err
                                best = str(line).replace("\\", "")
                    except:
                        pass
        try:
            if 'acceleration' in path:
                models = json.loads(''.join(best.split('t')[1] + best.split('t')[2]).replace("\'", "\"")) # If contains acceleration model
                models[0]['name'] = "acceleration_approach" # If contains acceleration model
            else:
                models = json.loads(best.split('t')[1].replace("\'", "\"")) # If not contains acceleration model
            subjects = [int(subj_id)]
            sim, trials = Cohen_movObst2(subjects)
            sim.reset()
            sim.models = models
            sim.simulate_all(trials, t_start='obst_onset', t_end='obst_out', ps='trial')
            errors[subj_id] = sim.test('p_dist', all_errors=True)
            print(f'{subj_id} finished')
            print(f'filename {filename}')
            print(f'models {sim.models}')
            print(f'error {np.mean(errors[subj_id])}')
        except Exception as e:
            print(e)
    return errors

    
errors = cross_validate('Cohen_movObst2_all_cohen_avoid1&2&3&4&heading_only_obst_onset')
# Compute BIC
for key, val in errors.items():
    print(key, np.mean(val))
'''BIC = n*ln(MSE) + k*ln(n))'''
mses = [x for a in errors.values() for x in a]
mses = [x**2 for x in mses]
n = len(mses)
BIC = n * np.log(np.mean(mses)) + k * np.log(n)
print(f'The BIC of model {sim.models[1]["name"]} is {BIC}, MSE is {np.mean(mses)}, k is {k}, n is {n}')

In [ ]:
'''Output errors for analysis in R'''
with open('Cohen_movObst2_CV_errors_obst_onset.csv', 'a') as f:
    f.write('model,subject,error\n')
for i in ['', '2', '3', '4']:
    model = 'cohen_avoid' + i
    errors = cross_validate('Cohen_movObst2_cross_validation_cohen_avoid' + i + '_obst_onset')
    with open('Cohen_movObst2_CV_errors_obst_onset.csv', 'a') as f:
        for subj, error in errors.items():
            for err in error:
                log = [model, subj, err]
                log = [str(x) for x in log]
                f.write(','.join(log) + '\n')
            

In [ ]:
'''Minimum Passing Distance'''
%matplotlib qt
load_data('Cohen_movObst2_data')
sim = sim2
fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.5), constrained_layout=True)
ax.set_title('Avoidance Model 1')
ax.set_ylabel('SMPD (m)')
ax.set_xlabel('normalized time (%)')
ax.set_ylim((-3, 3))
last_smpd_front = []
last_smpd_behind = []
for j, i in enumerate(sim.i_trials):
    t0 = data.info['obst_onset'][i]
    t1 = data.info['obst_out'][i]
    p0 = sim.p_pred[j]
    p1 = data.info['p_obst'][i][t0:t1]
    v0 = sim.v_pred[j]
    v1 = data.info['v_obst'][i][t0:t1]
    t = np.linspace(0, 100, len(p0))
    smpd = []
    for _p0, _p1, _v0, _v1 in zip(p0, p1, v0, v1):        
        smpd.append(min_sep(_p0, _p1, _v0, _v1)[0])
    ax.plot(t, smpd, 'k', linewidth=0.1, alpha=0.5)
    if smpd[-1] > 0:
        last_smpd_front.append(smpd[-1])
    else:
        last_smpd_behind.append(smpd[-1])
print(np.mean(last_smpd_front), np.std(last_smpd_front), np.mean(last_smpd_behind), np.std(last_smpd_behind))

### Bai_movObst1

In [ ]:
'''Simulate one trial avoidance'''
load_data('Bai_movObst1_data_30Hz')
############
# 518, 1679， 376， 647
i = 647
approach = approaches['Bai_movObst1b']['fajen_approach']['differential_evolution'][-1]
avoid = avoidances['Bai_movObst1']['cohen_avoid']['differential_evolution']['-1dtheta*dpsi_onset']
model_name = 'Avoidance model 1'
############
%matplotlib qt
models = [approach, avoid]
sim = ODESimulator(data=data, ref=[0, 1])
sim.models = models
sim.simulate_all(trials=[i], t_start='dtheta*dpsi_onset', t_end='obst_out', ps='trial')
dist_error = sim.test('p_dist')
heading_error = sim.test('phi_RMSE') * 180 / np.pi
speed_error = sim.test('s_RMSE')
t_length = (sim.data.info['obst_out'][i] - sim.data.info['obst_onset'][i]) / sim.data.Hz
title = model_name + \
        ', subj ' + str(sim.data.info['subj_id'][i]) + \
        ', trial ' + str(sim.data.info['trial_id'][i]) + \
        ', obst path angle: ' + str(sim.data.info['obst_angle'][i]) + ' °' + \
        ', obst speed: ' + str(sim.data.info['obst_speed'][i]) + ' m/s' + \
        ',\n distance error: ' + str(round(dist_error, 2)) + ' m' + \
        ', heading RMSE: ' + str(round(heading_error, 2)) + ' °' + \
        ', speed RMSE: ' + str(round(speed_error, 2)) + ' m/s'
print(f'Simulation time {t_length} s, distance error {dist_error} m')
sim.play(linestyles=['--', '-', '-', '-'], plot=True, fontsize=13, xrange=[-5, 5], yrange=[-5.5, 5.5],
         title=title, fig_size=[10, 3.5], save=False)

In [ ]:
'''Simulate all trials model 1'''
############
# subject = 1
approach = approaches['Bai_movObst1b']['fajen_approach']['differential_evolution'][-1]
avoid = avoidances['Bai_movObst1']['cohen_avoid2']['differential_evolution']['-1dtheta*dpsi_onset']
############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim1, trials1 = Bai_movObst1(subjects)
sim1.reset()
sim1.models = models
sim1.simulate_all(trials1, t_start='dtheta*dpsi_onset', t_end='obst_out', ps='trial')
print(f'mean distance error is {sim1.test("p_dist")}, sd is {np.std(sim1.test("p_dist", all_errors=True))}')
print(f'order accuracy is {sim1.test("order_accuracy")}')
print(f'phi RMSE is {sim1.test("phi_RMSE")}')
print(f'speed RMSE is {sim1.test("s_RMSE")}')
# BIC
errors1 = sim1.test('p_dist', all_errors=True)
k = len(models[1]) - 2
# n is the n_subj * n_conditions
n_angle = len([x for x in set(data.info['obst_angle']) if x and abs(x) != 180])
n_speed = len([x for x in set(data.info['obst_speed']) if x])
n = len(set(data.info['subj_id'])) * n_angle * n_speed
# n = len(sim1.p_pred)
MSE = np.mean(sim1.test('p_dist', all_errors=True)**2)
BIC = n * np.log(MSE) + k * np.log(n)
print(f'The BIC of model {sim1.models[1]["name"]} is {BIC}, MSE is {MSE}, k is {k}, n is {n}')


In [ ]:
len([x for x in set(data.info['obst_speed']) if abs(x) != 0])

In [ ]:
'''Simulate all trials model 2'''
load_data('Bai_movObst1_data_30Hz')
############
# subject = 1
approach = approaches['Bai_movObst1b']['fajen_approach2']['differential_evolution'][-1]
avoid = avoidances['Bai_movObst1']['cohen_avoid4']['differential_evolution']['-1dtheta*dpsi_onset']

############
%matplotlib qt
models = [approach, avoid]
subjects = range(20)
sim2, trials2 = Bai_movObst1(subjects)
sim2.reset()
sim2.models = models


sim2.simulate_all(trials2, t_start='dtheta*dpsi_onset', t_end='obst_out', ps='trial')
errors2 = sim2.test('p_dist', all_errors=True)
print(f"distance error is {np.mean(errors2)} sd is {np.std(errors2)}")
print(f"order accuracy is {sim2.test('order_accuracy')}")
print(f"heading RMSE is {sim2.test('phi_RMSE')}")
print(f"speed RMSE is {sim2.test('s_RMSE')}")

# Compute BIC
'''BIC = n*ln(MSE) + k*ln(n))'''
k = len(models[1]) - 2
# n is the n_subj * n_conditions
n_angle = len([x for x in set(data.info['obst_angle']) if x and abs(x) != 180])
n_speed = len([x for x in set(data.info['obst_speed']) if x])
n = len(set(data.info['subj_id'])) * n_angle * n_speed
# n = len(sim2.p_pred)
MSE = np.mean(sim2.test('p_dist', all_errors=True)**2)
BIC = n * np.log(MSE) + k * np.log(n)
print(f'The BIC of model {sim2.models[1]["name"]} is {BIC}, MSE is {MSE}, k is {k}, n is {n}')

In [ ]:
'''Compare two simulations'''
%matplotlib qt
plt.plot(errors1, label=sim1.models[1]['name'])
plt.plot(errors2, label=sim2.models[1]['name'])
plt.legend()
plt.xlabel('trial#')
plt.ylabel('distance error (m)')
for i in range(len(errors1)):
    if errors1[i] - errors2[i] > 0.08 and errors2[i] < 0.05:
        print(sim1.i_trials[i], errors1[i], errors2[i])

In [ ]:
'''Signed Minimum Predicted Distance'''
%matplotlib qt
sim = sim2
fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.5), constrained_layout=True)
ax.set_title('Avoidance model 1')
ax.set_ylabel('SMPD (m)')
ax.set_xlabel('normalized time (%)')
ax.set_ylim((-3, 3))
last_smpd_front = []
last_smpd_behind = []
for j, i in enumerate(sim.i_trials):
    t0 = data.info['dtheta*dpsi_onset'][i]
    t1 = data.info['obst_out'][i]
    p0 = sim.p_pred[j]
    p1 = data.info['p_obst'][i][t0:t1]
    v0 = sim.v_pred[j]
    v1 = data.info['v_obst'][i][t0:t1]
    t = np.linspace(0, 100, len(p0))
    smpd = []
    for _p0, _p1, _v0, _v1 in zip(p0, p1, v0, v1):        
        smpd.append(min_sep(_p0, _p1, _v0, _v1)[0])
    x0 = smpd[0]
    for j, x1 in enumerate(smpd):
        if abs(x1 - x0) > 0.5:
            smpd[j-1:] = np.sign(smpd[j-1]) * np.abs(smpd[j-1:])
            break
        x0 = x1
    ax.plot(t, smpd, 'k', linewidth=0.1, alpha=0.5)
    if smpd[-1] > 0:
        last_smpd_front.append(smpd[-1])
    else:
        last_smpd_behind.append(smpd[-1])
print(np.mean(last_smpd_front), np.std(last_smpd_front), np.mean(last_smpd_behind), np.std(last_smpd_behind))

In [ ]:
load_data('Bai_movObst1_data_30Hz')

def cross_validate(folder):
    path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Results', folder))
    filenames = [os.path.join(path, name) for name in os.listdir(path) if name[-3:] == 'txt']
    errors = {}
    for filename in filenames:
        with open(filename, 'rb') as f:
            best = 0
            e_min = float('inf')
            if filename[-6] == '_' or filename[-6] == '-':
                subj_id = int(filename[-5])
            elif filename[-6] == 'l':
                subj_id = -1
            else:
                subj_id = int(filename[-6:-4])
            for i, line in enumerate(f):
                if i > 10:
                    try:
                        err = str(line).split("\\t")[2][:10]
                        if err[0] == '0':
                            err = float(err)
                            if err < e_min:
                                e_min = err
                                best = str(line).replace("\\", "")
                    except:
                        pass
        try:
            if 'acceleration' in path:
                models = json.loads(''.join(best.split('t')[1] + best.split('t')[2]).replace("\'", "\"")) # If contains acceleration model
                models[0]['name'] = "acceleration_approach" # If contains acceleration model
            else:
                models = json.loads(best.split('t')[1].replace("\'", "\"")) # If not contains acceleration model
            subjects = [int(subj_id)]
            sim, trials = Bai_movObst1(subjects)
            sim.reset()
            sim.models = models
            sim.simulate_all(trials, t_start='dtheta*dpsi_onset', t_end='obst_out', ps='trial')
            errors[subj_id] = sim.test('p_dist', all_errors=True)
            print(f'{subj_id} finished')
            print(f'filename {filename}')
            print(f'models {sim.models}')
            print(f'error {np.mean(errors[subj_id])}')
        except Exception as e:
            print(e)
    return errors

    
errors = cross_validate('Bai_movObst1_cross_validation_cohen_avoid1_with_approach_trained_on_Bai_movObst1b_all_product_onset')


In [ ]:
errors


In [ ]:
'''Output errors for analysis in R'''
with open('Bai_movObst1_CV_errors_product_onset.csv', 'a') as f:
    f.write('model,subject,error\n')
for i in ['1', '2', '3', '4']:
    model = 'cohen_avoid' + i
    errors = cross_validate('Bai_movObst1_cross_validation_cohen_avoid' + i + '_with_approach_trained_on_Bai_movObst1b_all_product_onset')
    with open('Bai_movObst1_CV_errors_product_onset.csv', 'a') as f:
        for subj, error in errors.items():
            for err in error:
                log = [model, subj, err]
                log = [str(x) for x in log]
                f.write(','.join(log) + '\n')
            

In [ ]:
'''Animate certain trials'''
sim = sim
i_trial = 0
sim.play(i_trial=i_trial)
sim.test('p_dist', i_trials=[i_trial])

In [ ]:
sim.models

In [ ]:
'''Find trials in certain range of error'''
sim = sim1
errors = sim.test('p_dist', all_errors=True)
print(f'smallest error {np.argmin(errors)}')
for i, err in enumerate(errors):
    if 0.189 < err < 0.19:
        print(i)

In [ ]:
'''Speed and heading errors correct vs incorrect trials'''
from sklearn.metrics import mean_squared_error
sim = sim1
phi_MSEs = []
speed_MSEs = []
trials = []
for i in range(len(sim.v_pred)):
    j = sim.i_trials[i]
#     if sim.pass_order_pred[i] == sim.data.info['pass_order'][j]:
    v_subj = np.gradient(sim.p_subj[i], axis=0) * sim.data.Hz
    v_pred = sim.v_pred[i]
    s0, phi0 = v2sp(v_subj)
    s1, phi1 = v2sp(v_pred)
    speed_MSEs.append(mean_squared_error(s0, s1))
    phi_MSEs.append(mean_squared_error(phi0, phi1))
    trials.append(i)
print(sim.models)
print(f'{len(phi_MSEs)} trials in total')
print(f'RMSE on heading is {np.sqrt(np.mean(phi_MSEs))}')
print(f'RMSE on speed is {np.sqrt(np.mean(speed_MSEs))}')
print(f"Distance is {sim.test('p_dist', i_trials=trials)}")

In [ ]:
sim2.test('p_dist')

In [ ]:
print(len(p0), len(p1), len(sim.i_trials))

In [ ]:
'''Compute pred_order'''
s = sim
orders = []
for i in range(len(s.p_pred)):
    j = s.i_trials[i]
    true_order = s.data.info['pass_order'][j]
    p0 = s.p_pred[i][-1]
    v0 = s.v_pred[i][-1]
    p1 = s.p_obst[i][-1]
    _beta = beta(p0, p1, v0)
    angle = s.data.info['obst_angle'][j]
    pred_order = np.sign(_beta * -angle)
    orders.append(true_order == pred_order)
print(np.mean(orders))

In [ ]:
sim.test('order_accuracy')

In [ ]:
'''t-test between two models'''
err1 = sim.test('p_dist', all_errors=True)
err2 = sim2.test('p_dist', all_errors=True)
print(ttest_rel(err1, err2))
print(f'sd1 = {np.std(err1)}, sd2 = {np.std(err2)}')
print(f'df = {len(err1) - 1}')

In [ ]:
'''Show one trial from batch simulations'''
######## index in simulated trials only
i = 83
########
i_trial = sim.i_trials[i]
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(i, title=title, save=False)

# When beta and dpsi has the same sign it means pass in front, otherwise it means pass from behind
pass_order = sim.data.info['pass_order'][i]
pred = sim.pass_order_pred[i]
print('pass order ', pass_order, 'prediction ', pred)
print(f"err is {sim.test('p_dist', i_trial=i)}")

In [ ]:
'''plot true vs predicted speed by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0.5, 2)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    s_subj, s_pred = traj_speed(subj, Hz), traj_speed(pred, Hz)
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Speed
    axes[(abs(angle), speed)].plot(t, s_subj, 'r')
    axes[(abs(angle), speed)].plot(t, s_pred, 'b')


In [ ]:
'''plot true vs predicted heading by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0, 1.5)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    h_subj, h_pred = v2sp(np.gradient(subj, axis=0) * Hz)[1], v2sp(np.gradient(pred, axis=0) * Hz)[1]
    if h_subj[0] < 0:
        h_subj += pi
        h_pred += pi
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Heading
    axes[(abs(angle), speed)].plot(t, h_subj, 'r')
    axes[(abs(angle), speed)].plot(t, h_pred, 'b')
    if abs(angle) == 112.5 and speed == 1.2:
        print(i, i_trial)
